In [5]:
import os
import random
import numpy as np
import pandas as pd
from scipy import stats, interpolate

In [6]:
def fix_distribution(x, y):
    indices=[0]
    for i in range(1,len(y)):
        if y[i] != y[i-1]:
            indices.append(i)
    x, y = x[indices], y[indices]
    return x, y


def get_inverse(args, query, n_bins=100, log=False):
    kernel = stats.gaussian_kde(query.period.values)
    lower, upper = min(query.period.values), max(query.period.values)
    if args.log:
        x = np.logspace(np.log10(lower),np.log10(upper),args.n_bins)
    else:
        x = np.linspace(lower,upper,args.n_bins)
    y = np.cumsum(kernel(x))/np.sum(kernel(x))
    x, y = fix_distribution(x, y)
    try:
        spline = interpolate.CubicSpline(y, x)
    except ValueError:
        return None, None
    else:
        xnew = np.linspace(min(y),max(y),args.n_bins)
        return xnew, spline
    
# Main function to import when not using CLI
def get_period(teff, logg, period=[], path='rotation.csv', min_sample=20, res_teff=100., res_logg=0.1, log=False, n_bins=100, verbose=True):
    # read in known rotation periods and get limits
    df = pd.read_csv(path)
    for tt, ll in zip(teff, logg):
        per = np.nan
        # select stars near target in HR diagram
        query = df.query("teff >= %f and teff < %f and logg >= %f and logg < %f"%(tt-(res_teff/2.), tt+(res_teff/2.), ll-(res_logg/2.), ll+(res_logg/2.)))
        if len(query) < min_sample:
            if verbose:
                print('WARNING: not enough in the sample to create an accurate distribution.\nTry changing the resolution of the grid to include more stars!')
                print('Currently using teff +/- %.1f K and logg +/- %.2f dex'%(res_teff/2.,res_logg/2.))
        else:
            kernel = stats.gaussian_kde(query.period.values)
            lower, upper = min(query.period.values), max(query.period.values)
            if log:
                x = np.logspace(np.log10(lower),np.log10(upper),n_bins)
            else:
                x = np.linspace(lower,upper,n_bins)
            y = np.cumsum(kernel(x))/np.sum(kernel(x))
            x, y = fix_distribution(x, y)
            try:
                spline = interpolate.CubicSpline(y, x)
            except ValueError:
                continue
            else:
                # draw random number to map back to period distribution
                per = spline(random.random())+0.
        period.append(per)
    return np.array(period)

### For a single star
(needs to be in list-like form since that's how command line would read the parameters in)

In [7]:
prot = get_period([5777.], [4.4])
print(prot)

[17.07247756]


### For a handful of stars
**note:** make sure to reset the period array by including the keyword argument

In [12]:
teff = [5000., 5250., 6000., 5838., 5777.]
logg = [4.6, 4.4, 4.3, 4.4, 4.4]

prot = get_period(teff, logg, period=[])
print(prot)

[23.98999569 17.97997179 14.08481249 21.35882387 14.71273078]
